# Computing presentation: STAN

## Mamie Wang

### Oct 18, 2016

## STAN

* Imperative probabilistic programming language

* Domain-specific

* A Stan program defines a statistical model through a conditional probability function $p(\theta \lvert y, x)$,

    where $\theta =$ sequence of modeled unknown values (e.g. model parameters, latent variables, missing data, future prediction),
    
    and $y =$ a sequence of modeled known values,
    
    and $x =$ a sequence of unmodeled predictors and constraints (hyperparameters)

[comment]: # (imperative: based on assignment, loop, conditionals, local variables, object-level function application, and array-like data structure, just like C++)
[comment]: # (latent variables are as opposed to the observed variables, which are variables not directly observed, but rather inferred from other variables that are obesrved)
[comment]: # (hyperparameters are parameters of prior distribution, used to distinguish them from parameters of the model for the underlying system under analysis. For example, if one is using a beta distribution to model the distribution of the parameter p of Bernoulli distribution. The p is a parameter of the underlying system, and alpha, beta are the hyperparameters.)
[resources]: # (https://en.wikipedia.org/wiki/Hyperparameter, file:///Users/mwang/Downloads/stan-reference-2.12.0.pdf)


### Typical workflow of statistical modeling using STAN 
* Example of a latent-mixture model

#### Problem statement

A group of 15 people sit an exam made up of 40 true-or-false questions, and they get 21, 17, 21, 18, 22, 31, 31, 34, 34, 35, 35, 36, 39, 36, and 35 right. 


It seems that the first 5 people might just be guessing, but the last 10 at least had some level of knowledge. 


[comment]: # (A finite mixture model of an outcome assume that the outcome is drawn from one of serveral distributions, controlled by a categorial mixing distribution)
[resource]: # (Bayesian Cognitive Modeling: a practical course)

#### Make some assumptions 

* Two different groups of people with different probabilities of success
* Assume that the guessing group having a success probability of 0.5, and the knowledge group having a probability greater than 0.5
* To simplify the model, no individual differences within group.
* The goal is to infer which group each person belongs and the rate of success of the knowledge group.

#### Construct the model
![Graphical Model](graphicalModel.PNG "Graphical model for inferring memebrship of two latent groups, with different rates of success in answering exam questions (Lee & Wagenmakers, 2013, Fig.6.1, p.77)")
* $n$ = number of total exam questions (40)
* $k_i$ = the number of correct answers for the $i$th person
* $\theta_i$ = probability of success on each question for $i$th person, 
    where $\theta = \psi$ for guessing group,
    and $\theta = \phi$ for knowledge group.
* $z_i$ = the binary indictor variable for the group each person belongs (0: guessing group, 1: knowledge group). 
* Assumptions about priors
    * $z_i$ is equally likely to be 0 or 1 => $z_i$ ~ Bernoulli(1/2)
    * $\psi = 1/2$
    * $\phi$ ~ Uniform(0.5, 1)
    * $k_i$ ~ Binomial($\theta_i$, $n$)

[comments]: # (Discrete variables are designated by square nodes, continous by circle nodes. White nodes are latent and have to be inferred. Grey nodes are manifest; either observed data or set as constants. Single bordered nodes are stochastic and double bordered are deterministic derived variables.)

* Implementation in Stan

In [60]:
library(rstan)
options(warn=-1)
model <- "
// Exam Scores
data { 
  int<lower=1> p;
  int<lower=0> k[p];
  int<lower=1> n;
}
transformed data {
  real psi;
  // First Group Guesses
  psi <- .5;
}
parameters {
  // Second Group Has Some Unknown Greater Rate Of Success
  real<lower=.5,upper=1> phi; 
} 
transformed parameters {
  vector[2] lp_parts[p];
  // Data Follow Binomial With Rate Given By Each Person's Group Assignment
  for (i in 1:p) {
    lp_parts[i,1] <- log(.5) + binomial_log(k[i], n, phi);
    lp_parts[i,2] <- log(.5) + binomial_log(k[i], n, psi); 
  }
}
model {
  for (i in 1:p)
    // log_sum_exp(x,y) = log(exp(x) + exp(y))
    // increment_log_prob(x) log(p(x))= logZ + x
    increment_log_prob(log_sum_exp(lp_parts[i]));  
} 
generated quantities {
  int<lower=0,upper=1> z[p];
  
  for (i in 1:p) {
    vector[2] prob;
    prob <- softmax(lp_parts[i]);
    z[i] <- bernoulli_rng(prob[1]);
  }
}"

k <- c(21, 17, 21, 18, 22, 31, 31, 34, 34, 35, 35, 36, 39, 36, 35)
p <- length(k)  # number of people
n <- 40  # number of questions

data <- list(p=p, k=k, n=n) # to be passed on to Stan

myinits <- list(
  list(phi=.75))
  
parameters <- c("phi", "z")  # parameters to be monitored

# The following command calls Stan with specific options.
samples <- stan(model_code=model,   
                data=data, 
                init=myinits,  # If not specified, gives random inits
                pars=parameters,
                iter=20000, 
                chains=1, 
                thin=1,
                # warmup = 100,  # Stands for burn-in; Default = iter/2
                # seed = 123  # Setting seed; Default is random seed
)
# Summary of the posterior distribution
options(warn=0)
print(samples, digits=3)


SAMPLING FOR MODEL '52c0a5630f330045657b34fc40ad480b' NOW (CHAIN 1).

Chain 1, Iteration:     1 / 20000 [  0%]  (Warmup)
Chain 1, Iteration:  2000 / 20000 [ 10%]  (Warmup)
Chain 1, Iteration:  4000 / 20000 [ 20%]  (Warmup)
Chain 1, Iteration:  6000 / 20000 [ 30%]  (Warmup)
Chain 1, Iteration:  8000 / 20000 [ 40%]  (Warmup)
Chain 1, Iteration: 10000 / 20000 [ 50%]  (Warmup)
Chain 1, Iteration: 10001 / 20000 [ 50%]  (Sampling)
Chain 1, Iteration: 12000 / 20000 [ 60%]  (Sampling)
Chain 1, Iteration: 14000 / 20000 [ 70%]  (Sampling)
Chain 1, Iteration: 16000 / 20000 [ 80%]  (Sampling)
Chain 1, Iteration: 18000 / 20000 [ 90%]  (Sampling)
Chain 1, Iteration: 20000 / 20000 [100%]  (Sampling)
 Elapsed Time: 0.534411 seconds (Warm-up)
               0.528149 seconds (Sampling)
               1.06256 seconds (Total)

Inference for Stan model: 52c0a5630f330045657b34fc40ad480b.
1 chains, each with iter=20000; warmup=10000; thin=1; 
post-warmup draws per chain=10000, total post-warmup draws=10000.

#### Summary statistics
* 95%-posterior-high-density region (i.e. the shortest interval on a posterior density for some given confidence level) of the phi-parameter is 0.828-0.894. 
* the latent class indicator z indicates the class membership without any error

## Side notes: MCMC vs. variational Bayes


* MCMC gives a numerical approximation of the true posterior  by constructing a reversible Markov-chain that has as its equilibrium distribution the target posterior distribution.
    * Pros: asymptotically exact
    * Cons: hard to determine if convergence happens, time consuming for large dataset, hard to perform sensitivity analysis

[1]: https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo "MCMC"
[2]: # (Both can be used to provide an approximation for posterior probability density of unobserved variables.)

### Variational Inference
* The basic idea is to replace a complicated posterior $p(y | x)$ with a simpler model $q(y)$ (so that we only need to model distribution of y for the x we actually observed)
* Faster: variational Bayes approximates the posterior analytically (but it provides a locally-optimal solution).
* Sensitivity analysis: A local robustness measure can be easily derived analytically


[2]: http://www.cs.jhu.edu/~jason/tutorials/variational.html "A high level explanation of variational inference"

References:
* Giordano, Ryan, Tamara Broderick, and Michael Jordan. "Robust Inference with Variational Bayes." arXiv preprint [arXiv:1512.02578](https://arxiv.org/abs/1512.02578) (2015).
* Giordano, Ryan, et al. "Fast robustness quantification with variational Bayes." arXiv preprint [arXiv:1606.07153](https://arxiv.org/abs/1606.07153) (2016).
* [GitHub Repo](https://github.com/rgiordan/MVNMixtureLRVB) for LRVB fitting of multivariate normal mixture model
